In [16]:
### Import all the libraries ###
import requests 
import re
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import tensorflow as tf 
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
#We use the wordnet to find the synonym of the word. If the synonym are the same for some words, then there is a high chance that the text is talking about this subject. The synonyms of a word are returned as a nested list of synonyms of the different senses of the input word in the given language, since these different senses are not mutual synonyms#
nltk.download('wordnet')
#We use the omw package with the stopswords#
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
################################

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nelso\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\nelso\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nelso\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\nelso\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\nelso\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\nelso\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-d

True

In [17]:
### Create a input to ask wichpage we want to analyse###
name = input('Quel est la page a consulte ?')

Quel est la page a consulte ?Barack Obama


In [18]:
#################################################################
#### Scrapp the page of the given search  to give categories #####
#################################################################
def get_categories(name):
    url = "https://fr.wikipedia.org/wiki/" + name.replace(" ", "_")
    #requete get on the main url page
    page = requests.get(url)
    #verification of http code
    if page.status_code == 200:
        #parsing the page
        soup = BeautifulSoup(page.content, 'html.parser')
        #recuperation of the categories
        categories = soup.find_all('div', class_="mw-normal-catlinks")[0]
    else:
        #print an error if the page is unfindable
        print("Erreur lors de la requete")

    #creation of a empty list
    liste = []
    #recuperation of the <a> tag
    links = categories.find_all('a')
    #adding the content of the <a> tags to the list
    for link in links:
        liste.append(link.text)
    liste.remove("Catégories")
    return(liste)
# exemple of the execution of the function
get_categories(name)

['Barack Obama',
 'Président des États-Unis au XXIe siècle',
 'Mémorialiste américain',
 'Mémorialiste du XXe siècle',
 'Mémorialiste du XXIe siècle',
 'Avocat en Illinois',
 "Sénateur des États-Unis pour l'Illinois",
 "Membre du Sénat de l'État de l'Illinois",
 'Candidat à la présidence des États-Unis désigné par le Parti démocrate',
 'Élection présidentielle américaine de 2008',
 'Élection présidentielle américaine de 2012',
 'Personnalité liée à Chicago',
 'Politique à Chicago',
 'Personnalité du Parti démocrate (États-Unis)',
 'Personnalité du Parti démocrate en Illinois',
 "Personnalité liée à la guerre d'Irak",
 'Personnalité politique afro-américaine',
 'Personnalité gauchère',
 "Victime d'une tentative de meurtre",
 "Étudiant de l'université Columbia",
 'Étudiant de la faculté de droit de Harvard',
 "Étudiant de l'Occidental College",
 "Professeur à l'université de Chicago",
 "Docteur honoris causa de l'université Northwestern",
 "Docteur honoris causa de l'université de Notre-

In [19]:
#############################################################################
#### Scrapp the page of the given search to give the user who contribute #####
#############################################################################
def get_user_page(name):
    # URL but this time we replace the space by underbar, because of the url system
    url = 'https://fr.wikipedia.org/w/index.php?title='+name.replace(" ", "_")+'&action=history&offset=&limit=500'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    # Getting of all users who contribute to the page, the maximum of printable user contribution is 500 user
    users = soup.find_all('a', {'class':'mw-userlink'})

    list_user = []
    #adding the user to the list
    for user in users:
        if user.text not in list_user:
            list_user.append(user.text)
    return(list_user)

get_user_page(name)

['Bédévore',
 'Harmonide',
 'PouX',
 'Pautard',
 'Vexa805',
 'FélixFR',
 'Philotam',
 'Polymagou',
 'Simans9093',
 'Solfado',
 'OrlodrimBot',
 'Do not follow',
 'Eximau',
 '2607:fa49:4103:d300:cd9:9aea:99db:c588',
 'Malaria28',
 '37.165.59.126',
 '37.165.86.50',
 'Jean-Christophe BENOIST',
 'Leskheys',
 'JLM',
 '23.91.90.210',
 'Synoman Barris',
 '176.166.23.68',
 'Harrieta171',
 '62.235.213.202',
 'D952',
 '194.30.53.82',
 '66.131.96.112',
 '2a01:e0a:978:b6f0:c893:f4ac:6bc:48db',
 'Nicolas22g',
 'Cdigat',
 '85.26.56.197',
 'Lomita',
 'Susoko',
 'Da Kong King',
 '94.109.184.95',
 'Alexisdepris',
 '90.102.22.130',
 '81.240.80.193',
 'Mahaef',
 '2a04:cec0:1185:c5b9:246b:8627:e5a2:a0f2',
 'Alexcadotte',
 '2a09:1cc4:c:6d00:757a:7bd5:874b:6ff',
 'Criric',
 'LeFrançais2022',
 'Cheep',
 'Korectot',
 'Ciaran.london',
 'Qdesjo',
 'Film sur Léo Major',
 'TwoWings',
 'Malik2Mars',
 "Jack Rabbit Slim's",
 'Maxfr03',
 'Jpor 7',
 'Sthubertliege',
 'WikiCleanerBot',
 'ContributorQ',
 'Kaftabac',
 'Tr

In [20]:
#####################################################################################################################################
#### Scrapp the page of the user and returning the db with the 5 pages that they contribute and how much they contribute to them #####
#####################################################################################################################################
def sumup(name):
    # Creating the db with panda
    data = {'User_Name':  [],
        'Five_most_used_Theme': [],
        'Five_most_used_Theme_number': [],
        }
    # calling the function to get the name of the contributer
    list_user = get_user_page(name)
    # Modifing the url to get the user contribution
    liste_underscore = [x.replace(" ", "_") for x in list_user]
    liste_plus = [x.replace(" ", "+") for x in list_user]
    df_local = pd.DataFrame(data)
    
    #
    for i in range(len(list_user)):
        url = 'https://fr.wikipedia.org/w/index.php?title=Sp%C3%A9cial:Contributions/'+liste_underscore[i]+'&target='+liste_plus[i]+'&offset=&limit=500'
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        
        # We count the number of iteration in the user contribution page
        page_list = []
        counter = {}
        for link in soup.find_all('a', {'class': 'mw-contributions-title'}):
            if 'Discussion' not in link.text and 'Wikipédia' not in link.text:
                element = link.text
                if element in counter:
                    counter[element] += 1
                else:
                    counter[element] = 1
        # We organize the number of article and select the 5 most updated page
        sorted_counter = dict(sorted(counter.items(), key=lambda item: item[1], reverse=True))
        top_five = dict(list(sorted_counter.items())[:5])
        df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
    return df_local

df = sumup(name)


C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_N

C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_N

C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_N

C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_N

C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_N

C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_N

C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_N

C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_N

C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_N

In [21]:
#exemple of the database of a website, with 5 users, there top 5 contribution name and their count
df.head()

,User_Name,Five_most_used_Theme,Five_most_used_Theme_number
0,Bédévore,"[Gerard Troost, Angelo Heilprin, Catégorie:Dra...","[6, 3, 2, 2, 2]"
1,Harmonide,[Convention internationale pour la répression ...,"[7, 5, 5, 4, 4]"
2,PouX,"[Vincent Bolloré, Partmaximum, Guy Georges, Sy...","[5, 4, 4, 4, 4]"
3,Pautard,"[Tiaret, Birdpaula, Thermes d'Aulus-les-Bains,...","[3, 3, 2, 2, 2]"
4,Vexa805,"[Saison 4 de Stranger Things, Noah Schnapp, Fi...","[53, 41, 28, 18, 15]"


In [22]:
# We are doing it for another page
name = input('Quel est la page a consulte ?')
categories = get_categories(name)
df2 = sumup(name)

Quel est la page a consulte ?Emmanuel Macron


C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_N

C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_N

C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_N

C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_N

C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_N

C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_N

C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': list(top_five.keys()), 'Five_most_used_Theme_number': list(top_five.values())}, ignore_index=True)
C:\Users\nelso\AppData\Local\Temp\ipykernel_18216\1736470100.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({'User_N

In [23]:
# We are printing the second table
df2.head()

,User_Name,Five_most_used_Theme,Five_most_used_Theme_number
0,WikiCleanerBot,"[Projet:Correction syntaxique/Analyse 573, Pro...","[2, 2, 2, 2, 2]"
1,Lapin du Morvan,"[Franck Riester, Liste d'accidents nucléaires,...","[28, 19, 15, 15, 12]"
2,Tom10tom,"[Fusillade des émeutes de Kenosha, Emmanuel Ma...","[18, 16, 15, 13, 11]"
3,Groupir !,"[Utilisateur:Groupir !/Fantomas, Le Grand Rest...","[159, 65, 36, 24, 17]"
4,RobokoBot,"[Forum économique mondial, Liste des anciennes...","[4, 4, 4, 3, 3]"


In [24]:
# We are taking the commun user of both table and printing the new table to show the possible in commun user and their 'passion'
def inner_join(df1,df2):
    return(pd.merge(df1, df2, on='User_Name', how='inner').drop(['Five_most_used_Theme_number_y', 'Five_most_used_Theme_y'], axis=1).rename(columns={'Five_most_used_Theme_x': 'Five_most_used_Theme', 'Five_most_used_Theme_number_x': 'Five_most_used_Theme_number'}))
dfmerged = inner_join(df,df2)
dfmerged

,User_Name,Five_most_used_Theme,Five_most_used_Theme_number
0,Bédévore,"[Gerard Troost, Angelo Heilprin, Catégorie:Dra...","[6, 3, 2, 2, 2]"
1,Pautard,"[Tiaret, Birdpaula, Thermes d'Aulus-les-Bains,...","[3, 3, 2, 2, 2]"
2,Philotam,[Ordre de succession à l'ancien trône de Portu...,"[50, 38, 21, 18, 17]"
3,Polymagou,"[William Richard Tolbert, Emma Shannon Walser,...","[5, 4, 4, 3, 3]"
4,OrlodrimBot,"[Projet:Histoire/Articles récents, Portail:Éta...","[7, 7, 7, 6, 6]"
5,Do not follow,"[Kurt Sitte, Harold James, Diana Joyce Croft, ...","[19, 11, 9, 9, 6]"
6,JLM,"[François Maréchal, Selex ES, Daniel Cohn-Bend...","[22, 12, 12, 11, 8]"
7,D952,"[Fusillade du 23 décembre 2022 à Paris, Avatar...","[18, 5, 4, 3, 3]"
8,Lomita,"[Saint-Vincent-de-Tyrosse, Utilisateur:Kizilas...","[5, 5, 5, 4, 3]"
9,LeFrançais2022,"[Emmanuel Macron, Élisabeth II, Abdelaziz Bout...","[40, 13, 12, 12, 11]"
